In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -r 'drive/My Drive/kogpt2_novel_gen/requirements.txt'

     |████████████████████████████████| 256kB 11.8MB/s 
     |████████████████████████████████| 68.7MB 78kB/s 
     |████████████████████████████████| 1.1MB 52.0MB/s 
     |████████████████████████████████| 675kB 52.2MB/s 
     |████████████████████████████████| 3.8MB 26.9MB/s 
     |████████████████████████████████| 890kB 46.8MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.9.1-cp36-cp36m-linux_x86_64.whl size=470033 sha256=8d2cfe7c2be2d678edefe4b055ff2e28bc6d906b89f7055080f082aefb1f3c09
  Stored in directory: /root/.cache/pip/wheels/af/60/16/1f8a40e68b85bd9bd7960e91830bca5e40cd113f3220b7e231
  Created wheel for kss: filename=kss-1.3.1-cp36-cp36m-linux_x86_64.whl size=251538 sha256=ad87ac34a931dd59588788cae687371d7025f8653f6c25aaeb6a5b921c244f3a
  Stored in directory: /root/.cache/pip/wheels/8b/98/d1/53f75f89925cd95779824778725ee3fa36e7aa55ed26ad54a8
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=3e44dfa14ef511f217957227e79152c9f

In [ ]:
!pip install git+https://github.com/SKT-AI/KoGPT2.git@master

  Cloning https://github.com/SKT-AI/KoGPT2.git (to revision master) to /tmp/pip-req-build-i28x3y0w
  Running command git clone -q https://github.com/SKT-AI/KoGPT2.git /tmp/pip-req-build-i28x3y0w
  Created wheel for kogpt2: filename=kogpt2-0.1.1-cp36-none-any.whl size=14054 sha256=f64587958b85c67c3cde0877055c755f0f743f93a0d429c7f0ac7a393b1ba38f
  Stored in directory: /tmp/pip-ephem-wheel-cache-oncq4ac_/wheels/44/82/d2/27d3a318967141f892c13b49930333491aa0992b08d9f478e2
Successfully built kogpt2


In [ ]:
!pip install gensim

In [ ]:
import numpy as np

Rouge Start


In [ ]:
def custom_rouge1(target, infer, mode):
  # Intersect of Unigram / total len(target)
  if mode == "recall":
    total_len = len(target.split(" "))
  if mode == "precision":
    total_len = len(infer.split(" "))


  count1 = 0
  count2 = 0
  pre_1 = ""
  pre_2 = ""
  for w1 in target.split(" "):
    bi_1 = pre_1 + " " + w1
    pre_1 = w1
    for w2 in infer.split(" "):
      bi_2 = pre_2 + " " + w1
      pre_2 = w2
      if w1 == w2 : 
        count1 += 1
      if bi_1 == bi_2 :
        count2 += 1

  return count1/total_len, count2/total_len
  

In [ ]:
def cacl_bleu(origin, mymodel, novel):
  encoding = "utf-8"
  
  
  origin_dict = {}
  f = open(origin, "r", -1, encoding)
  for line in f.readlines():
    _key = line.split(",")[0]
    _sent = ",".join(line.split(",")[2:]).strip()
    origin_dict[_key] = _sent
  f.close()


  mymodel_dict = {}
  f = open(mymodel, "r", -1, encoding)
  for line in f.readlines():
    #line = line.replace("......", "")
    _key = line.split(",")[0]
    _sent = ",".join(line.split(",")[2:]).strip()
    mymodel_dict[_key] = _sent
  f.close()

  

  base = []
  my =  []
  for key, sent in novel.items():
    for sent_novel in sent:
      if len(sent_novel) != 0:
        
        #Origin 
        # A = min(1, len(origin_dict[key].split(" "))/len(sent_novel.split(" ")))
        # B = calc_bleu_B(sent_novel, origin_dict[key])

        # score = A*B

        # base.append(score)
 
        A = min(1, len(mymodel_dict[key].split(" "))/len(sent_novel.split(" ")))
        B = calc_bleu_B(sent_novel, mymodel_dict[key])

        score = A*B

        my.append(score)

        # score1, score2 = custom_rouge1(sent_novel, mymodel_dict[key], mode)


  return base, my

In [ ]:
import math
from collections import Counter

In [ ]:
def calc_bleu_B(sent_novel, prev):
  count = [0,0]#,0,0]
  len_count = [0,0]#,0,0]

  ref_list = sent_novel.split(" ")
  ref_list = [x for x in ref_list if x != ""]
  pred_list = prev.split(" ")
  pred_list = [x for x in pred_list if x != ""]

  for NGRAM in range(1, 1 + len(count)):
    ref_wordDict = Counter()
    pred_wordDict = Counter()
    matched_list = []

    for idx_ref in range(0, len(ref_list) - NGRAM + 1):
      ref_gram = ref_list[idx_ref:idx_ref + NGRAM]
      ref_gram = " ".join(ref_gram)
      
      ref_wordDict[ref_gram] += 1 

      for idx_pred in range(0, len(pred_list) - NGRAM + 1):
        pred_gram = pred_list[idx_pred:idx_pred + NGRAM]
        pred_gram = " ".join(pred_gram)

        pred_wordDict[pred_gram] += 1 
      
      print(ref_gram, pred_gram)
      if ref_gram == pred_gram:
        matched_list.append(ref_gram)
        count[NGRAM - 1] += 1

    for matched_word in matched_list:
      corr = pred_wordDict[matched_word] - ref_wordDict[matched_word]
      count[NGRAM - 1] = count[NGRAM - 1] - corr
    # for ref_key, ref_count in ref_wordDict.items():
    #   print(ref_key, ref_count)
    #   print(pred_wordDict[ref_key])
    # print("REF")
    # print(ref_wordDict)
    # print("PRED")
    # print(pred_wordDict)

    len_count[NGRAM - 1] += len(pred_list) - NGRAM

  #print(count)

  score = 1
  for i in range(2):
    score = score*count[i]/len_count[i]
  
  score = math.sqrt(score)
  return score


In [ ]:
base, my = cacl_bleu("/content/drive/MyDrive/kogpt2_novel_gen/201112/SCORE/new_argmax_basemodel.txt", 
                                            "/content/drive/MyDrive/kogpt2_novel_gen/201112/SCORE/new_argmax_mymodel.txt", 
                                        sent_dict)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
것 같았다. 안은 채
는 채
사람이 채
족히 채
삼, 채
사 채
백 채
명은 채
되는 채
것 채
같았다. 채
는 사람이 안은 채
사람이 족히 안은 채
족히 삼, 안은 채
삼, 사 안은 채
사 백 안은 채
백 명은 안은 채
명은 되는 안은 채
되는 것 안은 채
것 같았다. 안은 채
는 채
걸 채
싫어하는 채
것 채
같았다. 채
고왜승속(高矮僧俗) 채
수 채
백명이 채
진중 채
는 걸 안은 채
걸 싫어하는 안은 채
싫어하는 것 안은 채
것 같았다. 안은 채
같았다. 고왜승속(高矮僧俗) 안은 채
고왜승속(高矮僧俗) 수 안은 채
수 백명이 안은 채
백명이 진중 안은 채
에 채
모여 채
있어서 채
일시에 채
각자의 채
얼굴을 채
자세히 채
볼 채
수 채
없었다. 채
에 모여 안은 채
모여 있어서 안은 채
있어서 일시에 안은 채
일시에 각자의 안은 채
각자의 얼굴을 안은 채
얼굴을 자세히 안은 채
자세히 볼 안은 채
볼 수 안은 채
수 없었다. 안은 채
일까 채
봐 채
잔뜩 채
겁을 채
먹고 채
있는 채
것 채
같았다. 채
일까 봐 안은 채
봐 잔뜩 안은 채
잔뜩 겁을 안은 채
겁을 먹고 안은 채
먹고 있는 안은 채
있는 것 안은 채
것 같았다. 안은 채
같았다. 채
그 채
순간 채
가슴이 채
아파 채
오더니 채
왁! 채
하고 채
소리를 채
내며 채
한 채
같았다. 그 안은 채
그 순간 안은 채
순간 가슴이 안은 채
가슴이 아파 안은 채
아파 오더니 안은 채
오더니 왁! 안은 채
왁! 하고 안은 채
하고 소리를 안은 채
소리를 내며 안은 채
내며 한 안은 채
모금의 채
선혈을 채
토해 채
내었다. 채
모금의 선혈을 안은 채
선혈을 토해 안은 채
토해 내었다. 안은 채
혀 채
경력이 채
없어서 채
마치 채
살짝 채
스치는 채
것 채
같았다. 채
위일소의 채
한빙면 채
혀 경력이 안은 채
경력이 없어서 안은 채
없어서 마치 안은 채
마치 살짝 안은 채
살짝 스치는 안은 채
스치는 것 안은 채
것 같았다. 안은 

KeyboardInterrupt: ignored

In [ ]:
print(np.mean(base))
print(np.mean(my))

0.003476411534051227
0.0


In [ ]:
def cacl_rouge(origin, mymodel, novel, mode):
  origin_dict = {}

  #mode = "precision"
  #mode = "recall"

  encoding = "utf-8"
  f = open(origin, "r", -1, encoding)
  for line in f.readlines():
    _key = line.split(",")[0]
    _sent = ",".join(line.split(",")[2:]).strip()
    origin_dict[_key] = _sent
  f.close()


  mymodel_dict = {}
  f = open(mymodel, "r", -1, encoding)
  for line in f.readlines():
    line = line.replace("......", "")
    _key = line.split(",")[0]
    _sent = ",".join(line.split(",")[2:]).strip()
    mymodel_dict[_key] = _sent
  f.close()

  # tmp = []
  # for key, value in origin_dict.items():
  #   tmp.append(len(value))

  # print(np.mean(tmp))

  # tmp = []
  # for key, value in mymodel_dict.items():
  #   tmp.append(len(value))

  # print(np.mean(tmp))


  base_1, base_2 = [], []
  my_1, my_2 = [], []
  for key, sent in novel.items():
    for sent_novel in sent:
      if len(sent_novel) != 0:
      # Join Method --> All 0
      #sent_novel = " ".join(sent)
     
        score1, score2 = custom_rouge1(sent_novel, origin_dict[key], mode)

        base_1.append(score1)
        base_2.append(score2)
        
        score1, score2 = custom_rouge1(sent_novel, mymodel_dict[key], mode)
        my_1.append(score1)
        my_2.append(score2)


  return base_1, base_2, my_1, my_2

In [ ]:
base_1, base_2, my_1, my_2 = cacl_rouge("/content/drive/MyDrive/kogpt2_novel_gen/201112/SCORE/new_conjuct_basemodel.txt", 
                                            "/content/drive/MyDrive/kogpt2_novel_gen/201112/SCORE/new_conjuct_mymodel.txt", 
                                        sent_dict, "precision")

In [ ]:
for each in [base_1, base_2, my_1, my_2]:
  print(np.mean(each))

0.03655896561970572
0.05188839360783377
0.025000295796368
0.034132157796888954


In [ ]:
target_bigram = ['수 있는', '로테슈 자작은', '한 번', '알 수', '수 있었다.', '에르기 공작은', '것 같았다.', '수 없었다.', '수 없는', '에르기 공작이', '할 수', '고개를 끄덕였다.', '하인리 왕자는', '로테슈 자작이', '엘리자 백작', '고개를 저었다.', '하인리 왕자가', '입을 열었다.', '어쩔 수', '좀 더', '수 있을', '카프멘 대공은', '한숨을 내쉬었다.', '두 사람은', '것 같은', '안 되는', '수가 없었다.', '수밖에 없었다.', '카를 후작은', '로테슈 자작의', '황후 폐하."']
sent_dict= sent_from_bigram("/content/drive/My Drive/kogpt2_novel_gen/uchun.txt", target_bigram)

In [ ]:
 for key, sent in sent_dict.items():
  for sent_novel in sent:
    if len(sent_novel) != 0:
      print(sent_novel)
      print("------------")

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
내쉬더니 별안간  소매를 걷어 붙여  야들야들한 팔을 드러냈다.
------------
장취산은 고개를 들지 않을  수 없었다. 그는 비로소 그녀의 왼
------------
팔에 작은 흑색  강표(鋼標) 세 개가 박혀  있는 것을 발견했다.
------------
무기를 사용할 수 없었다.
------------
다. 하지만  그 거센 파도도 그의 상념을 지워비릴 수 없었다.
------------
이것을 본 장취산은 놀라지  않을 수 없었다. 그는 장도가 실수
------------
한 것으로만 생각했다. 그런데  그의 생각이 사실과 다르다는 게
------------
이고 있음을 깨달았다. 그는 의아해 하지 않을 수 없었다.
------------
장취산은 그녀의 표정이 진지해서 거절할 수 없었다. 그가 자리
------------
에 앉자, 은소소는  다시 만면에 미소를 띄고  그에게 술을 권했
------------
수 없었다. 해변에 정박되어  있는 배의 돛대가 연달아 꺾여지고
------------
있는 게 아닌가! 돛대 위에는 모두 깃발이 걸려 있었다. 천응교,
------------
백구수는 더 이상 태연한 척할 수 없었다.
------------
혼비백산하지 않을 수 없었다.
------------
수 없었다. 그들이 배에 오르자 사손은 즉시 배를 촐발 시키라고
------------
손짓했다. 배에는 모두 열  일곱 명의 타수가 있었다. 그런데 타
------------
라, 그녀의 적극적인 태도에  차츰 동요되지 않을 수 없었다. 더
------------
우기 지금 이 순간 은소소의 부드러우면서도 감미로운 체취가 코
------------
소는 즉시 호흡 장애를 느끼며  외마디 신음조차  낼 수 없었다.
------------
장취산은 경악을 금치 못하고 전광석화처럼 몸을 날려 사손의 등
------------
낮이 길고 밤은 짧아  도무지 날짜를 계산할 수 없었다. 그들은

In [ ]:
sent_dict

{'것 같았다.': ['에 있는 것 같았다.',
  '',
  '고는 고색찬연하여 오래 전에  만들어진 것 같았다. 곽양은 금현',
  '(琴絃)을 한 번 어루만지고 나서 뜯기 시작했다. 그녀가 뜯는 곡',
  '이렇게 되자 곽양은 울화통이 터질 것 같았다. 부모님의 명성은',
  '이미 만천하에 알려죠, 설령  무림인이 아닌 일반 백성이라 해도',
  '것 같았다. 딸기코 노인 방천로가 황급히 만류를 했다.',
  '',
  '가 뒤틀리며 숨이 막히는 것 같았다. 딸기코 방천로는 사제가 당',
  '한 것에 저으기 놀랐지만,  겉으로는 여전히 히죽히죽 웃으며 말',
  '정말 무공에 대해  전혀 모르는 것 같았다.  그러니 그의 상황이',
  '오죽이나 아슬아슬해 보였겠는가!  그런데도 그의 둔해 보이면서',
  '것 같았다. 주위에 있는  사람들은 절로 등골이 오싹해지는 한기',
  '를 느꼈다.',
  '들어 있는 것  같았다. 그들은 한결같이 청색  짧은 바지 차림에',
  '챙이 짧은 삿갓을 쓰고 있었다. 당시 나라에서는 폭정(暴政)으로',
  '인 것 같았다. 유대암은  심상치 않은 예감이 뇌리를 스치자, 얼',
  '른 신법을 펼쳐 염효들의 뒤를 쫓아갔다.',
  '담감을 느끼는 것 같았다. 그래도 보도를 꼭 쥐고 있자, 살이 타',
  '들어가는 소리가 더욱 요란하게 들렸다.',
  '이제 거의 뼈속까지 타들어간 것 같았다.',
  '',
  '몇이 적중된 것 같았다. 이 틈을 타서 유대암은 이미 십여 장 밖',
  '으로 벗어날 수 있었다. 그는 비록 한 사람을 들고 있지만, 경공',
  '상대방은 횃불을 들고 있는 것 같았다.',
  '',
  '다. 칼의 무게는 최소한 백 근 남짓 될 것 같았다. 바로 많은 사',
  '람의 목숨을  앗아간 그 도룡보도였다.  해동청은 도조히 살아서',
  '것 같았다. 유대암은 대뜸 천응교가 떠올라 경각심을 높였다.',
  '',
  '상대방은 뭐라고 몇 마디  하는 것 같았다. 그러나 워낙 소리가',
 

In [ ]:
def sent_from_bigram(path, target_bigram):

    sent_dict = {}

    for key in target_bigram:
      sent_dict[key] = []

    f = open(path, "r",  -1, "utf-8")
    #wordDict = Counter()
    sentences = f.readlines()

    prev = ""

    sentences.append("")
    for idx, sentence in enumerate(sentences):
      sentence = sentence.strip()
      sentence = sentence.replace("\n", "").replace("      ", "")
      
      if (sentence.startswith("TALK")):
          if (sentence.find('"') != -1):
              tok = sentence.split('"')
              sentence = tok[-1]
      
      for word in sentence.split():
          bi = prev + " " + word
          prev = word
          
          if bi in target_bigram :
            #sent_dict[bi].append(sentences[idx - 1].strip().replace("\n", ""))
            sent_dict[bi].append(sentence) 
            sent_dict[bi].append(sentences[idx + 1].strip().replace("\n", ""))

        
    f.close()
    return sent_dict

Rouge End

In [ ]:
import torch
from kogpt2.pytorch_kogpt2 import get_pytorch_kogpt2_model, get_kogpt2_model
from gluonnlp.data import SentencepieceTokenizer
from kogpt2.utils import get_tokenizer

In [ ]:
import random
import torch
import torch.nn.functional as F

def beamSearch():
    None

def random_sampling(predict, vocab, k = 1024):
    # k개 중 랜덤으로 선택된 값을 반환. 사실상 top_k와 같은 기
    gen =[]

    probs, indexs = torch.topk(predict, k=k, dim=-1)
    probs = probs.squeeze().tolist()[-1]
    indexs = indexs.squeeze().tolist()[-1]

    for i in range(len(indexs)):
        gen.append((vocab.to_tokens(indexs[i]),probs[i]))
        # print('topk word and value: ', gen)

    rand_num = random.randint(0,k-1)
    gen_word = vocab.to_tokens(indexs[rand_num])

    return gen_word


def top_p(logits, vocab, threshold = 0.9):
    sorted_logits, sorted_indices = torch.sort(logits, descending=True)
    indexs = sorted_indices.tolist()

    sorted_softmax_logits = F.softmax(sorted_logits, dim=-1)
    cum_probs = torch.cumsum( sorted_softmax_logits, dim=-1)


    sorted_indices_to_remove = cum_probs > threshold
    top_p_index = 0

    # Top-p에 해당하는 index를 획득
    for i in range(len(sorted_indices_to_remove)):
        if sorted_indices_to_remove[i]== True:
            top_p_index = 0 if i==0 else i-1
            break

    # for i in range(top_p_index):
      # print('gen '+str(i)+': '+vocab.to_tokens(indexs[i]))

    rand_num = random.randint(0, top_p_index) # top-p 분포에서 랜덤 샘플링
    top_p_sample_num = indexs[rand_num]
    gen_word = vocab.to_tokens(top_p_sample_num)
    print('selected token: '+gen_word+ ' softmax value:'+str(sorted_softmax_logits[rand_num]))

    return gen_word

def top_k(predict, vocab, k):
    # topk 중 랜덤으로 선택된 값을 반환.
    gen = []

    probs, indexs = torch.topk(predict, k=k,dim=-1)
    # probs = probs.squeeze().tolist()[-1]
    # indexs = indexs.squeeze().tolist()[-1]
    probs = probs.tolist()
    indexs = indexs.tolist()

    for i in range(len(indexs)):
        gen.append((vocab.to_tokens(indexs[i]), probs[i]))
        # print('topk word and value: ', gen)

    rand_num = random.randint(0, k - 1)
    gen_word = vocab.to_tokens(indexs[rand_num])

    return gen_word

def is_conjunction(word):
    conjs = ['그리고', '그래서', '그러나', '갑자기', '분명히']
    if (word in conjs):
        return True
    
    return False

def conjunction_after_argmax(prev_word, predict, vocab, k):
    # 문장 시작 지점 topk 중 접속사가 있을 경우 접속사를 선택하고 아니면 argmax를 선택
    gen = []
    is_word_start = False
    
    if (prev_word.endswith('.')):
        is_word_start = True

    probs, indexs = torch.topk(predict, k=k,dim=-1)
    probs = probs.tolist()
    indexs = indexs.tolist()

    maxword = vocab.to_tokens(indexs[0])
    if (maxword == '▁'):
        return maxword
    
    if (is_word_start == False):
        return maxword
    
    for i in range(len(indexs)):
        word = vocab.to_tokens(indexs[i])
        if (is_conjunction(word)):
            print('predict: '+str(word)+' / '+str(probs[i]) + ' but argmax: '+str(maxword) + ' / ' + str(probs[0]))
            return word
    
    return maxword

In [ ]:
# Origin 
#load_path = '/content/drive/My Drive/kogpt2_novel_gen/augment test 201011/ORIGIN_checkpoint_201_72184_1602264713.tar'
# 200 Epochs Origin + 200Epochs Only Aug
#load_path = '/content/drive/My Drive/kogpt2_novel_gen/augment test 201011/ONLYAUG_checkpoint_only_augment_final_checkpoint_399_77557_1602282157.tar'
# DATA[Aug + Origin } 200 Epochs
#load_path = '/content/drive/My Drive/kogpt2_novel_gen/augment test 201011/ORIGIN_checkpoint_201_72184_1602264713.tar'
#load_path = '/content/drive/My Drive/kogpt2_novel_gen/checkpoint/checkpoint_201_72184_1602264713.tar'
load_path = '/content/drive/MyDrive/kogpt2_novel_gen/201112/checkpointsfinal_checkpoint_399_276075_1605331961.tar'

load_path ="/content/drive/MyDrive/kogpt2_novel_gen/checkpoint for augment test 2/checkpoint for augment test 2checkpoint_203_167164_1603338542.tar"

In [ ]:
### 4. Tokenizer
tok_path = get_tokenizer()
model, vocab = get_pytorch_kogpt2_model()
tok = SentencepieceTokenizer(tok_path,  num_best=0, alpha=0)

checkpoint = torch.load(load_path)
model.load_state_dict(checkpoint['model_state_dict'])

model.eval()

using cached model
using cached model
using cached model


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50000, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): Laye

In [ ]:
def denormalize_character(words):
    char_map = {
        '남주인공는': "<남자주인공>은", 
        '남주인공은': "<남자주인공>은", 
        '여주인공를': "<여자주인공>을",
        '여주인공을': "<여자주인공>을",
        '남주인공가': "<남자주인공>이",
        '남주인공이': "<남자주인공>이",
        '여주인공가': "<여자주인공>이",
        '여주인공이': "<여자주인공>이",
        '남주인공': "<남자주인공>",
        '여주인공': "<여자주인공>",
        '여주2': "<여자주인공2>",
        '스승': "<스승>"}
    
    for k,v in char_map.items():
        words = words.replace(k, v)
        
    return words
    
def inference(sent, mode):
    toked = tok(sent)
    output_size = 200 # 출력하고자 하는 토큰 갯수

    prev_word = ""
    
    for i in range(output_size):
        input_ids = torch.tensor([vocab[vocab.bos_token],]  + vocab[toked]).unsqueeze(0)
        #print(input_ids)
        pred = model(input_ids)[0]

        if (mode == "argmax"):
            gen = vocab.to_tokens(torch.argmax(pred, axis=-1).squeeze().tolist())[-1]
        elif (mode == "top_k"):
            last_pred = pred.squeeze()[-1]
            gen = top_k(last_pred, vocab, 5)
        elif (mode == "conjunction_after_argmax"):
            last_pred = pred.squeeze()[-1]
            gen = conjunction_after_argmax(prev_word, last_pred, vocab, 10)
            
        if gen == '</s>':
            break
        
        if (gen == '▁'):
            sent += ' '
        else:
            prev_word = gen.replace('▁', ' ')
            sent += prev_word
            
        toked = tok(sent)
    
    return denormalize_character(sent)

In [ ]:
denormalize_character("남주인공는 여주인공를 안고 비록 마음은 초조했지만")

'<남자주인공>은 <여자주인공>을 안고 비록 마음은 초조했지만'

In [ ]:
sent = '다시'

In [ ]:
inference(sent, 'argmax')

0
[768]
tensor([[  0, 768]])
0
[768, 15533]
tensor([[    0,   768, 15533]])
0
[768, 15533, 268]
tensor([[    0,   768, 15533,   268]])
0
[768, 15533, 268, 699]
tensor([[    0,   768, 15533,   268,   699]])
0
[768, 15533, 268, 699, 47571]
tensor([[    0,   768, 15533,   268,   699, 47571]])
0
[768, 15533, 268, 699, 47571, 47930]
tensor([[    0,   768, 15533,   268,   699, 47571, 47930]])
0
[768, 15533, 268, 699, 47571, 47930, 2622]
tensor([[    0,   768, 15533,   268,   699, 47571, 47930,  2622]])
0
[768, 15533, 268, 699, 47571, 47930, 2622, 128]
tensor([[    0,   768, 15533,   268,   699, 47571, 47930,  2622,   128]])
0
[768, 15533, 268, 699, 47571, 47930, 2622, 128, 47440]
tensor([[    0,   768, 15533,   268,   699, 47571, 47930,  2622,   128, 47440]])
0
[768, 15533, 268, 699, 47571, 47930, 2622, 128, 47440, 235]
tensor([[    0,   768, 15533,   268,   699, 47571, 47930,  2622,   128, 47440,
           235]])
0
[768, 15533, 268, 699, 47571, 47930, 2622, 128, 47440, 235, 47524]
tensor([

'다시 봐도 참 잘생긴 얼굴이다. "여전히 순진하네. 이후 두 사람은 루테비타는 어디가 데이트하기 좋은지, 애인의 생일 선물은 무엇을 살지, 여러 가지 이야기를 나누며 여정을 즐겼다. 내 옆에 있던 테 제거 시스템이 다시 낮은 목소리로 말했다. 내 옆에 있던 테 제거 시스템이 다시 낮은 목소리로 말했다. 얼굴 반반한 사내를 발견하자 사내도 다시 한 번 얼굴이 불그스름해졌다. 그는 큰 소리로 말했다. 잘생긴 게 이렇게 닮은 건가? 하하. 사내 연애 한 번 못 해 본 여자가 이런 거 따질 게 뭐야. 하하. 사내 연애가 잘생긴 거보다 못생긴 게 훨씬 중요한 것 아닌가. 대화를하는 동안 두 사람은 걸어 다니고 있었고 Ms. Mulani가 기다리고있는 거실로 왔습니다. "이 자리에 대한 욕심에 눈이 멀어 그런 짓까지 벌이니까." 이미 지하 감옥의 남자를 통해 범인이 카릴린이란 것은 알고 있는 두 사람이었다. "비서   '

In [ ]:
inference(sent, 'top_k')

'황후 폐하." "......" "카프멘 대공. 카프멘 대공이 혼자서 동대제국을 누를 수 있을까요?" "대공도 혼자 못 가게 할 거예요." "난 잘 모르겠어요." "그렇겠죠." 카프멘 대공은 입술을                                                                                                                                                        악귀같이 차며 눈썹 윗입을  톡톡 까치발로 갈겨 버렸다.. 그치 \'하지만?\' 영리하다 싶던 하렘이 깨어난 것이다, 깊은 슬픔에 잠겼던 그녀는 자석처럼 탁 열리고, 그를 끌어안은 채 그의 머리를 두 눈으로 바라 보았다가 깨달았다.. 샤 미라보다 예쁘던 코방스타는 어쩌다가 그의 품에 안겨 이렇게 초라한 꼴이로 변했을까? 그는 이젠 자신을 사랑하는 여자가 한심하다 여겨지는 거였다! 그 꼴을 보다 못한 파란 눈은 무섭고 두려웠다! "하, 지, 지금 이게 어떻게 돼요?" 기절하기 전 커다란 손에서 끊임없이 흘러나오기 시작하는 목소리는 몹시 두려웠고, 두려웠는데...... 지독한 감정은 한 여자의 심기를 건드려 깊은 슬픔도 불러일으켰나 보았다, 그녀는 커다란 손을 치웠다 세게 끌어낸 그는 결국 얼굴을 감쌌지만. 놀라 뒷덜미로 붉게 달아 오른 얼굴이 드러났다.. "하하." 잠시 놀랐던 하녀가 웃음을 터뜨렸다 제 손길을 멈췄다 그러더니 놀란 표정을'

In [ ]:
inference(sent, 'conjunction_after_argmax')

'다시 봐도 참 잘생긴 얼굴이다. "여전히 순진하네. 이후 두 사람은 루테비타는 어디가 데이트하기 좋은지, 애인의 생일 선물은 무엇을 살지, 여러 가지 이야기를 나누며 여정을 즐겼다. 내 옆에 있던 테 제거 시스템이 다시 낮은 목소리로 말했다. 내 옆에 있던 테 제거 시스템이 다시 낮은 목소리로 말했다. 얼굴 반반한 사내를 발견하자 사내도 다시 한 번 얼굴이 불그스름해졌다. 그는 큰 소리로 말했다. 잘생긴 게 이렇게 닮은 건가? 하하. 사내 연애 한 번 못 해 본 여자가 이런 거 따질 게 뭐야. 하하. 사내 연애가 잘생긴 거보다 못생긴 게 훨씬 중요한 것 아닌가. 대화를하는 동안 두 사람은 걸어 다니고 있었고 Ms. Mulani가 기다리고있는 거실로 왔습니다. "이 자리에 대한 욕심에 눈이 멀어 그런 짓까지 벌이니까." 이미 지하 감옥의 남자를 통해 범인이 카릴린이란 것은 알고 있는 두 사람이었다. "비서   '

In [ ]:
def calc_new_score(sent):
  word_score = 0
  len_score = 0
  len_sent = len(sent.split("."))

  for line in sent.split("."):
    line = str(line) + "."
    len_score += 1/(abs(6.8 - len(line.split(" ")))**0.5)
    if "두 사람" in line:
        tmp = line.count("두 사람")
        word_score += tmp 
    if "소리" in line:
        tmp = line.count("소리")
        word_score += tmp 


  len_score = len_score/len_sent
  score = word_score + len_score
  return score

In [ ]:
def calc_new_score_len(sent):
  word_score = 0
  len_score = 0
  len_sent = len(sent.split("."))

  for line in sent.split("."):
    line = str(line) + "."
    len_score += 1/(abs(6.8 - len(line.split(" ")))**0.5)

  len_score = len_score/len_sent
  return len_score

In [ ]:
def calc_new_score_word(sent):
  word_score = 0
  len_score = 0
  len_sent = len(sent.split("."))

  for line in sent.split("."):
    line = str(line) + "."
    if "두 사람" in line:
        tmp = line.count("두 사람")
        word_score += tmp 
    if "소리" in line:
        tmp = line.count("소리")
        word_score += tmp 
  return word_score

In [ ]:

path = "/content/drive/My Drive/kogpt2_novel_gen/201112/SCORE/new_conjuct_mymodel.txt"

f = open(path, 'r', -1, 'utf-8')
len_score = 0
word_score = 0
for line in f.readlines():
  sent = ", ".join(line.split(", ")[2:])
  #print(calc_new_score(sent), calc_new_score_len(sent), calc_new_score_word(sent))
  len_score += calc_new_score_len(sent)
  word_score +=calc_new_score_word(sent)

f.close()

print(len_score/31, word_score/31)

0.614966250854344 2.3225806451612905


## Unigram Score

In [ ]:
def calc_new_score(sent):
  word_score = 0
  len_score = 0
  len_sent = len(sent.split("."))

  for line in sent.split("."):
    line = str(line) + "."
    len_score += 1/(abs(6.8 - len(line.split(" ")))**0.5)
    if "두 사람" in line:
        tmp = line.count("두 사람")
        word_score += tmp 
    if "소리" in line:
        tmp = line.count("소리")
        word_score += tmp 


  len_score = len_score/len_sent
  score = word_score + len_score
  return score

In [ ]:
origin = "/content/drive/My Drive/kogpt2_novel_gen/201112/conjuct_mymodel.txt"
new ="new_conjuct_mymodel.txt"

f = open(origin, 'r', -1, 'utf-8')
target = open(new, 'w', -1, 'utf-8')
for line in f.readlines():
  target.write(line.split(", ")[0])
  target.write(", ")
  target.write(str(calc_new_score(", ".join(line.split(", ")[2:]))))
  target.write(", ")
  target.write(", ".join(line.split(", ")[2:]))
  #target.write("\n")

f.close()
target.close()


In [ ]:
def calc_score(sent):
  score = 0
  len_sent = len(sent.split("."))

  for line in sent.split("."):
    line = str(line) + "."
    score += 1/(abs(6.8 - len(line.split(" ")))**0.5)
    if "두 사람" in line:
        tmp = line.count("두 사람")
        score += tmp 
        #score += 1/(abs(6.8 - len(line.split(" ")))**0.5)
    if "소리" in line:
        tmp = line.count("소리")
        score += tmp 
        score += 1/(abs(6.8 - len(line.split(" ")))**0.5)
    
  score /= len_sent
  return score

In [ ]:
path = "/content/drive/My Drive/kogpt2_novel_gen/augment test 201027/NEWScore1101/UNIGRAMSCORE_topk_oring_200Epochs.txt"
f = open(path, 'r', -1, 'utf-8')
total_score = 0
for _line in f.readlines():
  sent = ",".join(_line.split(",")[2:])
  score = calc_score(sent)
  total_score += score
  #print(score)

print(total_score/31)

f.close()

0.49923425010506217


In [ ]:
bigram = "/content/drive/My Drive/kogpt2_novel_gen/201112/total_novel_bigram.txt"
target = "conjuct_basemodel.txt"

encoding = "utf-8"
f = open(bigram, "r", -1, encoding)
target_f = open(target, "w", -1, encoding)
i = 0
for line in f.readlines():
  i += 1
  each_word = ",".join(line.split(",")[:-1])
  sent = inference(each_word, 'conjunction_after_argmax')
  each_score = calc_score(sent)

  target_f.write(str(each_word) + ", ")
  target_f.write(str(each_score) + ", ")
  target_f.write(str(sent) + "\n")

  if i > 30:
    break
  
target_f.close()
f.close()

In [ ]:
# Summary 
path = "/content/drive/My Drive/kogpt2_novel_gen/201112/SCORE/new_conjuct_mymodel.txt"

f = open(path, 'r', -1, encoding)
score = 0
for line in f.readlines():
  each_score = float(line.split(",")[1])
  score += each_score
f.close()

print(score/31)

2.937546896015634


## Cos Similarity

In [ ]:
from transformers import GPT2LMHeadModel
from numpy import dot
from numpy.linalg import norm
import tensorflow as tf
import numpy as np

In [ ]:
sent = "나는 말했다."
toked = tok(sent)
vocab[toked][0]


3918

In [ ]:
model.get_input_embeddings().weight[vocab[toked][0]].shape



torch.Size([768])

In [ ]:
def return_similarity(sent1, sent2, vocab, embed):
  #model.get_input_embeddings().weight[vocab[tok(w1)]].shape
  count = 0
  hight_count = 0
  score = 0
  for w1 in vocab[tok(sent1)]:
    for w2 in vocab[tok(sent2)]:
      count +=1 
      a = embed[w1]
      b = embed[w2]

      a = a.detach().numpy()
      b = b.detach().numpy()

      cos_sim = dot(a, b)/(norm(a)*norm(b))
      
      if cos_sim > 0.7 :
        hight_count += 1
      score += abs(cos_sim)


  return count, hight_count, score

In [ ]:
def get_cos_sim(a, b):
  counta = 0
  countb = 0
  sim_list = []
  for word1 in a:
    for word2 in b: 
      
      cos_sim = dot(word1, word2)/(norm(word1)*norm(word2))
      #print(cos_sim)
      sim_list.append(cos_sim)

  return sim_list



In [ ]:
def get_embed(sent1, vocab, embed):
  #model.get_input_embeddings().weight[vocab[tok(w1)]].shape
  list_embed = []
  for w1 in vocab[tok(sent1)]:
      a = embed[w1]
      a = a.detach().numpy()

      list_embed.append(a)


  return list_embed

In [ ]:
list_sent = ["무수문은 뒹굴 굴러 일어나서 수리를 쳐다보았다. 몹시 부러운 생각이 일었다.",
             "말하면서 두 눈이 뒤집히기 시작한다. 말도 더듬거렸다. 육무쌍은 부모가 무엇 때문에 저렇듯 심한 상처를 입은 상태에서도 다투는지를 몰라 울면서 소리만 질렀다.",
             "그녀는 양과,소용녀 부부와 화산(華山)에서 헤어진 후,  삼년이넘도록  그들의 소식을 전혀 듣지 못했다.",
             "그런데 이게 웬일인가? 각원대사는 온몸이 쇠사슬로  감겨져  있고, 또한  육중한 철통을 짊어진 상황이거늘 아무리 곽양이 소리를 내어 뛰어가도  그를 앞지를 수가 없었다.",
             "장무기는 이처럼 생각을 했다.  바로 이때 갑자기 왼쪽 멀리 떨어진 긴 풀숲 사이에서 경미한  숨소리가 몇 번 전해 왔다.",
             " 바로 이때 갑자기 뒤에서 땡!  땡! 하는 소리가 들려오며 세 사람이 쏜살같이 달려오고 있었다. 모두 백포를 들러 입고 있었고,중간에서 달려오고 있는 자의 키는 매우 컸고 한 편은 여자인 것을 볼 수 있었다.",
             " 유운사가 왼손으로 손짓을 하자 묘풍사, 휘월사 셋은 동시에 몸을 날려  금화파파 옆으로 접근했다.",
             "그녀는 전에 대승관영웅연회 석상에서 양과에게 장검을 주었는데 양과가 여러 사람 앞에서 그녀의 장검을 거절하고 받지 않아 크게 무안을당했던 적이 있었다.",
             "두 첩자는 고개를 숙여 읍을 한뒤 기뻐서 돌아갔다.몽고의 선봉대는 양양의 수군(守軍)과 교전을 벌이기도 전에 이미 섬멸을 당했던 것이다. 황용이 무대에 서서 이 소식을 개방의 제자들에게 알리자 그들은 환호성을 지르며 좋아했다.",
             "목소리는 한 사람이 내는 것이 나이고 사씨 형제가 한꺼번에 지른 것이었다.",
             "일시적으로 초목대사에게   속아서 그랬나  보다 생각했다.   그런데 지금  금나라 군사들까지 끼여드는 것을 보자 화가 치밀어 견딜 수가 없었다.",
             "소매를 날리며 아래층으로 내려갔다. 금나라 병사들은 동료가 쓰러지는 것을  보자 소란을 피우며 벌서 그 중 몇 사람이 창을 비껴 들고 구처기의 등을 향해  찔렀다.",
             "그래서 황용은 범려(范蠢)가 어떻게 월왕(越王)인 구천(句踐)을 도와 공을 세웠고, 그 뒤 은퇴해서  서시와 더불어 태호에서  노닐었다는 고사를 한바탕  들려주었다.",
             "그 어부가 고개를 끄덕이고석  잔술을 따라 마신 뒤  다시 권한다. 황용과  어부는 의기 투합했는지 피차 시사(詩詞)에 관한  얘기를 나누며 시간 가는 줄도  모른다.",
             "가슴이 철렁하며 상대를  깔보면 큰일난다는  생각이 뒤늦게  들어 조심스럽게  대들었다.",
             "곽정은 서둘러 파오를 나가 화쟁의 처소로 갔다.화쟁 공주는 어머니와 함께  거처하고 있었다. 그녀는 요  며칠 동안 표정이  밝고 환했다.",
             "영고는  그가 자기 쪽에 등을  보이고 있는 것을  보자 복수할 좋은  기회라 생각하고  품속에서 날카로운 칼을 꺼내 그의 등을 향해 찌르려고 했다.",
             "저야 벌써 세상을 등지고  출가한 사람인데 어찌 천하의  영웅과 앞뒤를 다툴  수 있겠습니까?",
             "모두가 악당이라고 했다. 단 한 명도 착한 사람을 죽인 일이 없다니, 과연  양심에 물어도 부끄러울   바 없지  않은가.",
             "구양봉은 키가 커 두  다리를 구부리고 합마공을  쓰고 있는데도 홍칠공보다  머리 하나가 더 컸다.",
             "양과는 소용녀의 상처가 매우 깊어서 시간을 지체하면 치료의 기회를잃을까 두려워 계속 초조해 했다. 법왕이 돌연 소리 질렀다.",
             "말할 필요도 없이 금침에는 독이 발라져 있었다. 보통 바늘로 눈을 찌르기만 해도 장님이 된다.",
             "그 때 동편에 있는 개방의 무리 가운데 8대 제자 한 명이 일어나 손을 쭉 내밀어 큰 조개를 입에 대고 큰소리로 불었다.",
             "양장로는 두 주먹을 모아 답례를 보내며 사람들의 박수 소리가 멈추기를 기다렸다가 큰소리로 말했다.",
             "오늘 이곳에 계신 모든 분들은 강호상에 이름을 날리는 사나이들로 각자 방주를 원하고 있으리라 생각합니다.",
             "법왕은 몇 장 뒤를 따라가다가 그녀가 갑자기 뛰어내리는 것을 보고 서둘러 몸을 날려 구하려고 했다.",
             "그와 법왕은 서로 매우 멀리 떨어져 있는데도 그가 뱉은 침은 바람소리를 은은히 일으키며 마치 하나의 철판처럼 곧장 법왕의 얼굴로 날아왔다.",
             "법왕은 무공이 비록 그녀보다 뛰어났지만 감히 맞서 다투지를 못 했다.",
             "어느 날엔가에 이르러서야 그들은 비로소 상대방이 없으면 결코 더 이상 살 수가 없고, 상대방이 자기의 목숨보다 백 배 천 배 중요하다는 것을 알게 되었다.",
             "소용녀는 처량하게 웃고는 낮게 말했다."

             ]

In [ ]:
embed = model.get_input_embeddings().weight

#total_base = []
total_mymodel = []
#total_origin = []
for sent in list_sent:
  a = get_embed(sent, vocab, embed)
  #total_base.append(a)
  total_mymodel.append(a)


In [ ]:
def get_cos_sim_compare_embedding(a, b):

  list_cos = []
  for i in range(10):
    sent1 = a[i]
    sent2 = b[i]

    for j in range(len(sent1)):
      w1 = sent1[j]
      w2 = sent2[j]
      
      cos_sim = dot(w1, w2)/(norm(w1)*norm(w2))   

      list_cos.append(cos_sim)

  return list_cos

In [ ]:
score_list = get_cos_sim_compare_embedding(total_origin, total_mymodel)
np.mean(score_list)

0.9910615

In [ ]:
print(score_list)

[0.9934155, 0.9966142, 0.9975014, 0.99741536, 0.99767834, 0.9955363, 0.9945036, 0.99760723, 0.9927428, 0.99579525, 0.9969789, 0.9974983, 0.99823236, 0.988449, 0.99522847, 0.996621, 0.9927022, 0.99347353, 0.99823236, 0.9950982, 0.9939998, 0.99501294, 0.9927581, 0.9933489, 0.99190634, 0.9853791, 0.99823236, 0.99500406, 0.995826, 0.9970748, 0.99753183, 0.99823236, 0.9919038, 0.997962, 0.99511236, 0.9972545, 0.9941548, 0.98896086, 0.99558115, 0.9949134, 0.9972009, 0.99727905, 0.98942953, 0.9924128, 0.99500585, 0.99253577, 0.99332714, 0.99402434, 0.9944402, 0.9965949, 0.99444824, 0.9963735, 0.9952864, 0.9972837, 0.99439865, 0.99789137, 0.99823236, 0.99587077, 0.99372584, 0.9962689, 0.99688935, 0.99825937, 0.9945003, 0.9967063, 0.99522847, 0.9971157, 0.9927882, 0.9962961, 0.9968759, 0.997459, 0.9971863, 0.9973383, 0.9939475, 0.9940745, 0.99688935, 0.9940342, 0.9952495, 0.99386084, 0.99487287, 0.9932547, 0.9928526, 0.990594, 0.9961338, 0.99647295, 0.99823236, 0.9948204, 0.9923591, 0.9930248, 

In [ ]:
np.mean(score_list)

0.98233247

In [ ]:
get_cos_sim(total[0],total[0])

1.0
-0.025303211
-0.1227632
0.084217824
-0.027578495
0.033885546
0.002214274
-0.045146648
0.060365055
0.006469292
-0.06600519
0.032583
-0.07423395
0.0632916
-0.025129698
0.045186132
-0.039417367
0.02729176
-0.07423395
-0.025303211
1.0000001
-0.03340137
-0.06675974
0.021397956
-0.009981254
-0.018329943
0.02392505
0.010345485
-0.028525613
0.034054525
0.06940464
-0.021306349
0.036813796
-0.05938538
0.07199263
0.027438914
0.024032418
-0.021306349
-0.1227632
-0.03340137
1.0
-0.0518892
0.01617903
-0.00094905635
0.0017303276
0.11070603
-0.08424968
0.006608758
0.11008166
-0.025793374
0.14656243
-0.042190444
0.13690348
-0.055417087
0.0335084
-0.11074407
0.14656243
0.084217824
-0.06675974
-0.0518892
1.0000001
0.052194957
0.24300112
0.06318847
-0.06917788
0.0068410914
0.1331425
-0.054235548
0.11682047
-0.08450501
0.04618748
-0.08203536
0.07126735
0.034145847
0.12305158
-0.08450501
-0.027578495
0.021397956
0.01617903
0.052194957
1.0
0.15665054
-0.007971615
0.041234925
-0.038443148
-0.02853363
0.05

[1.0,
 -0.025303211,
 -0.1227632,
 0.084217824,
 -0.027578495,
 0.033885546,
 0.002214274,
 -0.045146648,
 0.060365055,
 0.006469292,
 -0.06600519,
 0.032583,
 -0.07423395,
 0.0632916,
 -0.025129698,
 0.045186132,
 -0.039417367,
 0.02729176,
 -0.07423395,
 -0.025303211,
 1.0000001,
 -0.03340137,
 -0.06675974,
 0.021397956,
 -0.009981254,
 -0.018329943,
 0.02392505,
 0.010345485,
 -0.028525613,
 0.034054525,
 0.06940464,
 -0.021306349,
 0.036813796,
 -0.05938538,
 0.07199263,
 0.027438914,
 0.024032418,
 -0.021306349,
 -0.1227632,
 -0.03340137,
 1.0,
 -0.0518892,
 0.01617903,
 -0.00094905635,
 0.0017303276,
 0.11070603,
 -0.08424968,
 0.006608758,
 0.11008166,
 -0.025793374,
 0.14656243,
 -0.042190444,
 0.13690348,
 -0.055417087,
 0.0335084,
 -0.11074407,
 0.14656243,
 0.084217824,
 -0.06675974,
 -0.0518892,
 1.0000001,
 0.052194957,
 0.24300112,
 0.06318847,
 -0.06917788,
 0.0068410914,
 0.1331425,
 -0.054235548,
 0.11682047,
 -0.08450501,
 0.04618748,
 -0.08203536,
 0.07126735,
 0.034

In [ ]:
# Get ORIGIN_DOC
origin_doc = []

f = open(bigram, 'r', -1, 'utf-8')
idx = 0
count = 1000
for line in f.readlines():
  idx += 1
  origin_doc.append(line.split(",")[0])
  if idx == int(count):
    break
f.close()

In [ ]:
# model, vocab = get_pytorch_kogpt2_model()
# tok = SentencepieceTokenizer(tok_path,  num_best=0, alpha=0)
embed = model.get_input_embeddings().weight

bigram = "/content/drive/My Drive/kogpt2_novel_gen/201112/total_novel_bigram.txt"


model_path = '/content/drive/My Drive/kogpt2_novel_gen/201112/SCORE/new_conjuct_mymodel.txt'
mymodel_doc = []
f = open(model_path, 'r', -1, 'utf-8')
for line in f.readlines():
  line = line.replace("'", "")
  line = line.replace('"', "")
  tmp_sent = ", ".join(line.split(", ")[2:])
  mymodel_doc.append(tmp_sent)
f.close()

TOTAL_COUNT = 0
HIGT_COUNT = 0
SCORE = 0 
# #origin word by bigram top 10000
for sent1 in origin_doc:
  for sent2 in mymodel_doc:#my model
    TOTAL_COUNT += 1
    count, hight_count, score = return_similarity(sent1, sent2, vocab, embed)

    # For Test
    # HIGT_COUNT += hight_count
    # SCORE += score

    HIGT_COUNT += hight_count/count
    SCORE += score/count

print(TOTAL_COUNT, HIGT_COUNT, SCORE)

31000 185.78744894043052 1656.9542887198972


In [ ]:
model = GPT2LMHeadModel.from_pretrained('/content/drive/My Drive/kogpt2_novel_gen/201112/checkpointsfinal_checkpoint_399_276075_1605331961.tar')  # or any other checkpoint
# word_embeddings = model.transformer.wte.weight  # Word Token Embeddings 
# position_embeddings = model.transformer.wpe.weight  # Word Position Embeddings 

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
text_index = tokenizer.encode('man',add_prefix_space=True)
vector = model.transformer.wte.weight[text_index,:]


UnicodeDecodeError: ignored

In [ ]:
!pip install konlpy

     |████████████████████████████████| 19.4MB 1.4MB/s 
     |████████████████████████████████| 460kB 44.7MB/s 
     |████████████████████████████████| 92kB 10.5MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0


In [ ]:
from kogpt2.utils import download, tokenizer, get_tokenizer
from gluonnlp.data import SentencepieceTokenizer
import gluonnlp
import numpy as np
from gensim.models import Word2Vec

In [ ]:
import gensim
from konlpy.tag import Kkma
kkma = Kkma()

In [ ]:
def sentencePieceTokenizer():
  tok_path = get_tokenizer()
  sentencepieceTokenizer = SentencepieceTokenizer(tok_path)

  return sentencepieceTokenizer

sentencepieceTokenizer= sentencePieceTokenizer()

using cached model


In [ ]:
### SentencePieceTokenizer 확인해보니 이상해서 Konlpy 사용
def get_tokenized_from_bigram(path, count):
  TOTAL_TOKENIZED = []
  f = open(path, 'r', -1, 'utf-8')
  idx = 0
  for line in f.readlines():
    idx += 1
    TOTAL_TOKENIZED.append(kkma.morphs(line.split(",")[0]))
    if idx == int(count):
      break
  f.close()

  return TOTAL_TOKENIZED

def get_tokenized_from_score(path):
  TOTAL_TOKENIZED = []
  f_path = path
  f = open(f_path, 'r', -1, 'utf-8')
  for line in f.readlines():
    line = line.replace("'", "")
    line = line.replace('"', "")
    tmp_sent = ",".join(line.split(",")[2:])
    TOTAL_TOKENIZED.append((kkma.morphs(tmp_sent)))
  f.close()
  return TOTAL_TOKENIZED


In [ ]:
novel = "/content/drive/My Drive/kogpt2_novel_gen/201112/total_novel_bigram.txt"
origin_score = "/content/drive/My Drive/kogpt2_novel_gen/201112/SCORE/new_argmax_basemodel.txt"
mymodel_score = "/content/drive/My Drive/kogpt2_novel_gen/201112/SCORE/new_argmax_mymodel.txt"

novel_token = get_tokenized_from_bigram(novel, 10000)
origin_token = get_tokenized_from_score(origin_score)
mymodel_token = get_tokenized_from_score(mymodel_score)

In [ ]:
path = "/content/drive/My Drive/kogpt2_novel_gen/201112/untokenized_bm_data.txt"
f = open(path, 'r', -1, 'utf-8')
total_novel = []

for line in f.readlines():
  total_novel.append((kkma.morphs(line)))

f.close()

In [ ]:
print(len(novel_token))
print(len(sum(novel_token, [])))
print(len(origin_token))
print(len(set(sum(origin_token, []))))
print(len(mymodel_token))
print(len(set(sum(mymodel_token, []))))

100
388
31
812
31
1042


## Just My Corpus Model

In [ ]:
#total = novel_token + origin_token + mymodel_token
#total = origin_token + mymodel_token

total = total_novel+ novel_token + origin_token + mymodel_token

model = Word2Vec(sentences = total, size = 64, window = 3, min_count = 3, workers = 4, sg = 1)

word_vectors = model.wv
vocabs = word_vectors.vocab.keys()
word_vectors_list = [word_vectors[v] for v in vocabs]
print(word_vectors.most_similar("사람"))

#print(word_vectors.similarity(w1 = "문책", w2 = '사람'))

[('귀부인', 0.7896506190299988), ('가족', 0.7793822288513184), ('작자', 0.7743322849273682), ('국민', 0.7656616568565369), ('학생', 0.7597019672393799), ('인간', 0.7584216594696045), ('외의', 0.7563577890396118), ('녀석', 0.7522015571594238), ('귀족', 0.7448378801345825), ('손님', 0.7395405769348145)]


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


## PreTrainig Merge Version

In [ ]:
model = Word2Vec(sentences = total, size =200, window = 2, min_count=2, workers = 4, sg = 1)
model.wv.save_word2vec_format("test.model")
#model.load("/content/drive/My Drive/kogpt2_novel_gen/augment test 201027/ko.bin")
#model.intersect_word2vec_format(fname = "/content/drive/My Drive/kogpt2_novel_gen/augment test 201027/ko.bin", binary = True)

UnicodeDecodeError: ignored

In [ ]:
ko_model = gensim.models.Word2Vec.load('/content/drive/My Drive/kogpt2_novel_gen/augment test 201027/ko.bin')
ko_model.intersect_word2vec_format(fname = "/content/test.model", binary = True)

UnicodeDecodeError: ignored

In [ ]:
word_vectors = ko_model.wv
vocabs = word_vectors.vocab.keys()
word_vectors_list = [word_vectors[v] for v in vocabs]
print(word_vectors.most_similar("사람"))
print(word_vectors.similarity(w1 = "두", w2 = '사람'))

[('젊은이', 0.6494427919387817), ('여인', 0.6287257671356201), ('백성', 0.6063710451126099), ('포졸', 0.6043275594711304), ('죄인', 0.5960500836372375), ('선비', 0.5868039131164551), ('부녀자', 0.5654411315917969), ('죄수', 0.5639811754226685), ('구경꾼', 0.5620019435882568), ('손님', 0.5589558482170105)]
-0.032042198


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


## 평가

In [ ]:
list_score = []

word_by_word_novel_token = list(set(sum(novel_token, [])))
#tmp = list(set(sum(origin_token,[]))) ## ORIGINAL
tmp = list(set(sum(mymodel_token,[]))) ## MYMODEL
total_len = len(tmp)

#word_vectors = ko_model.wv
score = 0
count_hight = 0
for word in tmp:
  word_score = 0
  len_count = 0
  for word_novel in word_by_word_novel_token:
    
    try:
      each = word_vectors.similarity(w1 = word_novel, w2 = word) 
      word_score += abs(each)
      len_count += 1

      if each > 0.95 :
        count_hight +=1
        
    except:
      pass

  if len_count == 0:
    pass
  else :
    word_score = word_score/len_count
  score += word_score

print(score/len(tmp))
print(count_hight)
print(count_hight/total_len)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.3844427691418292
749
0.718809980806142


In [ ]:
print(len(tmp), len(word_by_word_novel_token))

1042 2208


In [ ]:
print(len(list(set(sum(origin_token,[])))))

EDA

In [ ]:
from os import listdir
from os.path import isfile, join
from collections import Counter
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [ ]:
def bigram_counter(path, concat=False):
    f = open(path, "r",  -1, "utf-8")
    wordDict = Counter()
    sentences = f.readlines()
    
    prev = ""
    
    concatedStr = ""
    
    for sentence in sentences:
        sentence = sentence.strip()
        
        if (sentence.startswith("TALK")):
            if (sentence.find('"') != -1):
                tok = sentence.split('"')
                sentence = tok[-1]
                
        if (concat == True and (sentence.endswith(".") == False and sentence.endswith("」") == False)):
            concatedStr += sentence
            continue
        
        if (concat):
            concatedStr += sentence
            sentence = concatedStr
            concatedStr = ""
        
        for word in sentence.split():
            bi = prev + " " + word
            wordDict[bi] += 1.
            
            prev = word
    
    if (len(concatedStr) > 0):
        for word in concatedStr.split():
            bi = prev + " " + word
            wordDict[bi] += 1.
            
            prev = word
            
    f.close()
    
    return wordDict

def bigram(path, target, concat=False):
    wordDict = bigram_counter(path, concat)
    
    f = open(target, "w", -1, "utf-8")
    for word, freq in wordDict.most_common():
        f.write(str(word)+","+str(freq)+"\n")
        
    f.close()

In [ ]:
bigram("/content/drive/My Drive/kogpt2_novel_gen/201112/untokenized_bm_data.txt", "total_novel_bigram.txt", True)